# Generatív hálózatok

A Rekurrens Neurális Hálózatok (RNN-ek) és azok kapuzott cellaváltozatai, mint például a Hosszú Rövid Távú Memóriacellák (LSTM-ek) és a Kapuzott Rekurrens Egységek (GRU-k), lehetőséget nyújtanak a nyelvi modellezésre, azaz képesek megtanulni a szavak sorrendjét, és előrejelzéseket adni a következő szóra egy sorozatban. Ez lehetővé teszi, hogy az RNN-eket **generatív feladatokra** használjuk, például egyszerű szöveggenerálásra, gépi fordításra, sőt akár képaláírások készítésére is.

Az előző egységben tárgyalt RNN architektúrában minden RNN egység a következő rejtett állapotot adta ki eredményként. Azonban hozzáadhatunk egy másik kimenetet is minden rekurrens egységhez, amely lehetővé teszi, hogy egy **sorozatot** adjunk ki (amely az eredeti sorozattal azonos hosszúságú). Továbbá használhatunk olyan RNN egységeket is, amelyek nem fogadnak bemenetet minden lépésnél, hanem csak egy kezdeti állapotvektort vesznek, és ezután egy kimeneti sorozatot generálnak.

Ebben a jegyzetfüzetben egyszerű generatív modellekre összpontosítunk, amelyek segítenek szöveget generálni. Az egyszerűség kedvéért építsünk egy **karakter-szintű hálózatot**, amely betűről betűre generál szöveget. Az edzés során szükségünk lesz egy szövegkorpuszra, amelyet betűsorozatokra bontunk.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Karakter szintű szókincs létrehozása

Ahhoz, hogy karakter szintű generatív hálózatot építsünk, a szöveget szavak helyett egyedi karakterekre kell bontanunk. A korábban használt `TextVectorization` réteg erre nem képes, így két lehetőségünk van:

* Kézzel betölteni a szöveget és manuálisan elvégezni a tokenizálást, ahogy [ebben az hivatalos Keras példában](https://keras.io/examples/generative/lstm_character_level_text_generation/) látható.
* A `Tokenizer` osztályt használni karakter szintű tokenizáláshoz.

Mi a második lehetőséget választjuk. A `Tokenizer` szavakra történő tokenizálásra is használható, így könnyen válthatunk karakter szintű és szó szintű tokenizálás között.

A karakter szintű tokenizáláshoz a `char_level=True` paramétert kell megadnunk:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Azt is szeretnénk, hogy egy speciális token jelölje a **sorozat végét**, amelyet `<eos>`-nek fogunk nevezni. Adjuk hozzá manuálisan a szókincshez:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Most pedig, hogy a szöveget számsorozatokká kódoljuk, használhatjuk:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Generatív RNN betanítása címek létrehozására

Az RNN betanításának módja a hírcímek generálására a következő. Minden lépésben veszünk egy címet, amelyet betáplálunk egy RNN-be, és minden bemeneti karakterhez megkérjük a hálózatot, hogy generálja a következő kimeneti karaktert:

![Kép, amely bemutatja az 'HELLO' szó generálását egy RNN segítségével.](../../../../../translated_images/hu/rnn-generate.56c54afb52f9781d.webp)

A szekvenciánk utolsó karakterénél megkérjük a hálózatot, hogy generálja a `<eos>` tokent.

A generatív RNN, amelyet itt használunk, fő különbsége az, hogy az RNN minden lépésének kimenetét felhasználjuk, nem csak az utolsó celláét. Ezt úgy érhetjük el, hogy az RNN cellának megadjuk a `return_sequences` paramétert.

Így a betanítás során a hálózat bemenete egy adott hosszúságú kódolt karakterekből álló szekvencia lesz, a kimenet pedig ugyanolyan hosszúságú szekvencia, de egy elemmel eltolva és `<eos>`-szal lezárva. Egy minibatch több ilyen szekvenciából fog állni, és **kitöltést** kell használnunk, hogy az összes szekvenciát igazítsuk.

Hozzunk létre olyan függvényeket, amelyek átalakítják számunkra az adathalmazt. Mivel a szekvenciákat minibatch szinten szeretnénk kitölteni, először az adathalmazt `.batch()` hívással csoportosítjuk, majd `map`-pel átalakítjuk. Így az átalakító függvény egy teljes minibatch-et fog paraméterként kapni:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Néhány fontos dolog, amit itt csinálunk:
* Először kinyerjük a tényleges szöveget a string tenzorból
* A `text_to_sequences` átalakítja a sztringek listáját egész szám tenzorok listájává
* A `pad_sequences` kitölti ezeket a tenzorokat a maximális hosszúságukig
* Végül egy-hot kódoljuk az összes karaktert, valamint elvégezzük az eltolást és a `<eos>` hozzáfűzést. Hamarosan meglátjuk, miért van szükségünk egy-hot kódolt karakterekre

Ez a függvény azonban **Pythonos**, azaz nem fordítható automatikusan Tensorflow számítási gráffá. Hibákat kapunk, ha megpróbáljuk közvetlenül használni ezt a függvényt a `Dataset.map` függvényben. Be kell csomagolnunk ezt a Pythonos hívást a `py_function` wrapper használatával:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Megjegyzés**: A Python-alapú és Tensorflow-alapú transzformációs függvények közötti különbségtétel elsőre túl bonyolultnak tűnhet, és talán azon gondolkodsz, miért nem alakítjuk át az adathalmazt standard Python függvényekkel, mielőtt átadnánk a `fit` függvénynek. Bár ez valóban lehetséges, a `Dataset.map` használatának óriási előnye van, mivel az adattranszformációs folyamat a Tensorflow számítási gráf segítségével történik, amely kihasználja a GPU számítási kapacitását, és minimalizálja az adatátvitelt a CPU és GPU között.

Most felépíthetjük a generátor hálózatunkat és elkezdhetjük a tanítást. Ez bármelyik rekurrens cellán alapulhat, amelyet az előző egységben tárgyaltunk (egyszerű, LSTM vagy GRU). Példánkban LSTM-et fogunk használni.

Mivel a hálózat karaktereket kap bemenetként, és a szókincs mérete viszonylag kicsi, nincs szükség beágyazási rétegre, az egy-hot kódolt bemenet közvetlenül az LSTM cellába kerülhet. A kimeneti réteg egy `Dense` osztályozó lesz, amely az LSTM kimenetét egy-hot kódolt token számokká alakítja.

Ezenkívül, mivel változó hosszúságú szekvenciákkal dolgozunk, használhatunk `Masking` réteget, hogy létrehozzunk egy maszkot, amely figyelmen kívül hagyja a szöveg kitöltött részeit. Ez nem feltétlenül szükséges, mivel nem igazán érdekel minket minden, ami a `<eos>` tokenen túl van, de a tapasztalatszerzés érdekében használni fogjuk ezt a rétegtípust. Az `input_shape` `(None, vocab_size)` lesz, ahol a `None` a változó hosszúságú szekvenciát jelzi, és a kimeneti alakzat szintén `(None, vocab_size)`, ahogy azt a `summary` mutatja:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Kimenet generálása

Most, hogy betanítottuk a modellt, szeretnénk használni, hogy kimenetet generáljunk. Először is szükségünk van egy módszerre, amely dekódolja a token számok sorozatával reprezentált szöveget. Ehhez használhatnánk a `tokenizer.sequences_to_texts` függvényt; azonban ez nem működik jól karakter szintű tokenizálás esetén. Ezért a tokenizálóból (amit `word_index`-nek hívunk) veszünk egy token szótárt, készítünk egy fordított térképet, és megírjuk a saját dekódoló függvényünket:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Most kezdjük el a generálást. Elindulunk egy `start` nevű sztringgel, amelyet egy `inp` nevű szekvenciává kódolunk, majd minden lépésben meghívjuk a hálózatunkat, hogy megállapítsuk a következő karaktert.

A hálózat kimenete, `out`, egy `vocab_size` elemű vektor, amely az egyes tokenek valószínűségeit reprezentálja. A legvalószínűbb token számát az `argmax` segítségével találhatjuk meg. Ezután hozzáadjuk ezt a karaktert a generált tokenek listájához, és folytatjuk a generálást. Ez a folyamat, amely során egy karaktert generálunk, `size` alkalommal ismétlődik, hogy előállítsuk a szükséges számú karaktert, és korábban befejezzük, ha az `eos_token` megjelenik.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Mintavételezés az edzés során

Mivel nincsenek hasznos metrikáink, mint például *pontosság*, az egyetlen módja annak, hogy lássuk, javul-e a modellünk, az az, ha **mintát veszünk** az edzés során generált szövegből. Ehhez **visszahívásokat** (callbacks) fogunk használni, vagyis olyan függvényeket, amelyeket átadhatunk a `fit` függvénynek, és amelyek az edzés során időszakosan meghívásra kerülnek.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Ez a példa már most is elég jó szöveget generál, de többféleképpen tovább javítható:

* **Több szöveg**. Csak címeket használtunk a feladatunkhoz, de érdemes lehet teljes szövegekkel is kísérletezni. Ne feledd, hogy az RNN-ek nem túl jók a hosszú szekvenciák kezelésében, ezért érdemes lehet azokat rövidebb mondatokra bontani, vagy mindig egy előre meghatározott `num_chars` értékű fix szekvenciahosszon tanítani (például 256). Próbáld meg a fenti példát ilyen architektúrává alakítani, a [hivatalos Keras oktatóanyag](https://keras.io/examples/generative/lstm_character_level_text_generation/) inspirációjával.

* **Többrétegű LSTM**. Érdemes lehet 2 vagy 3 rétegű LSTM cellákat kipróbálni. Ahogy az előző egységben említettük, az LSTM minden rétege bizonyos mintákat von ki a szövegből, és karakter-szintű generátor esetén elvárható, hogy az alsóbb LSTM szint a szótagokért, a magasabb szintek pedig a szavakért és szókapcsolatokért feleljenek. Ezt egyszerűen megvalósíthatod az LSTM konstruktorának rétegszám-paraméterének átadásával.

* Érdemes lehet kísérletezni **GRU egységekkel** is, hogy lásd, melyik teljesít jobban, valamint **különböző rejtett réteg méretekkel**. A túl nagy rejtett réteg túltanuláshoz vezethet (például a hálózat pontosan megtanulja a szöveget), míg a kisebb méret nem biztos, hogy jó eredményt hoz.


## Lágy szöveggenerálás és hőmérséklet

Az előző `generate` definícióban mindig a legnagyobb valószínűségű karaktert választottuk ki következő karakterként a generált szövegben. Ennek eredményeként a szöveg gyakran "ismételte" ugyanazokat a karakter-szekvenciákat újra és újra, mint ebben a példában:
```
today of the second the company and a second the company ...
```

Azonban, ha megnézzük a következő karakter valószínűségi eloszlását, előfordulhat, hogy a legnagyobb valószínűségek közötti különbség nem túl nagy, például egy karakter valószínűsége lehet 0.2, míg egy másiké 0.19, stb. Például, amikor a '*play*' szekvencia következő karakterét keressük, a következő karakter lehet egyaránt szóköz vagy **e** (mint a *player* szóban).

Ez arra a következtetésre vezet minket, hogy nem mindig "igazságos" a magasabb valószínűségű karaktert választani, mert a második legnagyobb valószínűségű karakter választása is értelmes szöveghez vezethet. Bölcsebb dolog **mintavételezni** a karaktereket az ideghálózat kimenete által adott valószínűségi eloszlásból.

Ez a mintavételezés a `np.multinomial` függvénnyel végezhető el, amely az úgynevezett **multinomiális eloszlást** valósítja meg. Az alábbiakban definiálva van egy függvény, amely ezt a **lágy** szöveggenerálást megvalósítja:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Bevezettünk egy újabb paramétert, amelyet **hőmérsékletnek** nevezünk, és amely azt jelzi, mennyire ragaszkodjunk a legnagyobb valószínűséghez. Ha a hőmérséklet 1,0, akkor tisztességes multinomiális mintavételt végzünk, és amikor a hőmérséklet végtelenhez közelít - minden valószínűség egyenlővé válik, és véletlenszerűen választjuk ki a következő karaktert. Az alábbi példában megfigyelhetjük, hogy a szöveg értelmetlenné válik, amikor túlzottan növeljük a hőmérsékletet, és "ciklusos" keményen generált szövegre hasonlít, amikor közelebb kerül a 0-hoz.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
